# Explore a NuGrid star model in iPython notebook

Modified for ASTR 404, Winter 2024. Execute this notebook on the [Astohub](https://astrohub.uvic.ca).


### Initialize session

In [ ]:
%pylab ipympl
from nugridpy import mesa as ms
from nugridpy import nugridse as mp

data_dir="/data/ASDR/NuGrid"  

ms.set_nugrid_path(data_dir)
mp.set_nugrid_path(data_dir)

### The MESA stellar evolution model

By default MESA is putting out two types of data. History data provides the time evolution of scalar quantities, one per time step. This data can be accessed with the `mesa.star_log` (or `mesa.history_data` which is the same) class.

MESA also outputs profile data at select time steps. Profiles are available via the `mesa_profile` class.

#### History data
Initialise the 2 solar-mass Z=0.02 MESA stellar evolution model from set1.2 using the seeker method:

In [ ]:
s=ms.star_log(mass=1,Z=0.02)   # or ms.history_data which is just an alias

`s` is an instance of the history data, which is the table of scalar quantities such as mass, $T_\mathrm{eff}$ etc for each time step or model.

There are a number of pre-defined functions or methods associated with this instances, such as an HRD or a Kippenhahn diagram:

In [ ]:
ifig=102;close(ifig);figure(ifig)
s.hrd_new()
legend(loc='lower right').draw_frame(False)

In [ ]:
ifig=107;close(ifig);figure(ifig)
s.kip_cont(ifig=ifig, boundaries=True,engenlevels=[1e2,1e6,1e12],modstart=100,modstop=500)

Note that each function has a pythonic doc string, e.g. `s.kip_cont`. 

The `s.cols` command gives a list of available columns in the history file. Maybe the most important method is `s.get('col_name)` which returns an array with the data in the column with name `col_name`.

Below we demonstrate this by funding a model similar to that of our sun (age $\approx 4.6$ billion years, mass $1\mathrm{M_\odot}$ and $Z=0.02$ and then find the central temperature and pressure of that stellar model.

In [ ]:
# this is the array with the ages of the stellar models
s.get('star_age')

In [ ]:
# find the index that corresponds to the approximate age of the sun
# recall that we initiated `s` as a 1Msun, Z=0.02 model
ind_sun = where(s.get('star_age')<4.6e9)[0][-1]
ind_sun

In [ ]:
# let's check what quantities we have
s.cols

As we can see we have the central temperature as well as some other central quantities already listed as scalar quantities in the history file:

```
'log_center_T': 123,
 'log_center_Rho': 124,
 'log_center_P': 125,
 'center_mu': 126,
 'center_ye': 127,
 'center_h1': 128
 ```
 
 Therefore knowing the central T and P is straight forward:

In [ ]:
s.get('log_center_T')[ind_sun]

In [ ]:
s.get('log_center_P')[ind_sun]

In [ ]:
s.get('center_h1')[ind_sun]

We can find the same results also by looking at profile data from the MESA output.

#### MESA profile files
The `mesa.mesa_profile` class provides access to the available profile data. We can again use the seeker method to just find the righ model by initial mass and metallicity. An additional argument `num=` specifies the model number. This behaviour can be changed, see the doc string for details. Profiles are not available for all models, and the method will report which nearby model it has found.

In [ ]:
# The `prof` instance is now holding the data of one profile for the given model number in the `num=` argument.
prof=ms.mesa_profile(mass=1,Z=0.02,num=ind_sun)

In [ ]:
# Again, have a look at the available quantieis
prof.cols

In [ ]:
# The data is presented in arrays that go from the outside inward
# therefore the last element is the central value
prof.get('pressure')[-1]

In [ ]:
# Of course we can also plot profiles
ifig=108;close(ifig);figure(ifig)
plot(prof.get('rmid'),prof.get('mu'))

In [ ]:
ifig=108;close(ifig);figure(ifig)
prof.plot('mass','c12',shape='-')

In [ ]:
# get all header attributes, these are some scalar quantites (like the ones in the history instance)
prof.header_attr